## Seismic model

### Function

In [1]:
from dwave.system import DWaveSampler, EmbeddingComposite # type: ignore


import numpy as np # type: ignore
import matplotlib.pyplot as plt # type: ignore
import sys, os, importlib

# Reload modules to ensure the latest changes are picked up
sys.path.append(os.path.abspath("modules"))
import modules.plotting
import modules.dwaveapi
import modules.gendata
importlib.reload(modules.plotting)
importlib.reload(modules.dwaveapi)
importlib.reload(modules.gendata)

from modules.dwaveapi import *
from modules.gendata import *
from modules.plotting import *

np.random.seed(99) # random noise
from tqdm import tqdm

### Running whole model

In [9]:
# Assuming the following functions are defined elsewhere:
# velocity_generator, device_location, find_intersections,
# remove_duplicate_intersections, calculate_distances,
# noise_generator, construct_Ad, binary_least_squares_qubo,
# dict_to_vector_auto, binary2real

def quantum_annealing_inversion(noise_level, results_dir='results30x10-new'):
    """
    Perform velocity inversion using quantum annealing with specified noise level.

    Parameters:
    noise_level (float): The noise level to be applied to travel time data.
    results_dir (str): Directory to store the results.

    Returns:
    None
    """
    # Define the velocity model
    rows, cols = 30, 10
    grid_size = (rows, cols)
    velocity_model = velocity_generator(rows, cols)

    # Iterate through layers
    for layer in tqdm(range(0, 15), desc="Process", colour='green'):
        # -----------------------------------------------------------------
        # Part 1 - Extract a part of the velocity model and set up geometry
        velocity_01 = velocity_model[layer, :]
        velocity_01 = velocity_01.reshape(1, -1)

        rows_new, cols_new = 1, 10
        grid_size_new = (rows_new, cols_new)

        z = device_location(n=20, rows=rows, new_min=0.1, new_max=29.9, linear=False)
        z = z - layer
        sources = [(0, i) for i in z]
        receivers = [(cols, i) for i in z]

        intersections = find_intersections(sources, receivers, grid_size=grid_size_new)
        unique_intersections = remove_duplicate_intersections(intersections)
        distances = calculate_distances(unique_intersections, grid_size=grid_size_new, sources=sources, receivers=receivers)

        # -----------------------------------------------------------------
        # Part 2 - Calculate D and T for least squares
        D = []
        T = []
        s1 = 1 / velocity_01

        nreceiver = len(receivers)
        nsource = len(sources)

        for i in range(nsource):
            for j in range(nreceiver):
                D.append(distances[:, :, j, i].flatten())
                T.append(sum(sum(distances[:, :, j, i] * s1)))
        D = np.array(D)
        s1 = s1.flatten()

        # -----------------------------------------------------------------
        # Part 3 - Add noise and remove zero elements
        T = np.array(T)
        indices_of_zero_np = np.where(T == 0)[0]

        # Add noise
        noise_percent = noise_generator(size=400, noise_level=noise_level)
        T_noise = T + noise_percent * T

        T_new = np.delete(T_noise, indices_of_zero_np, axis=0)
        D_new = np.delete(D, indices_of_zero_np, axis=0)

        # -----------------------------------------------------------------
        # Part 4 - Prepare parameters for quantum annealing
        M = D_new.copy()
        I = np.ones(M.shape[1])
        R = 3
        t = T_new.copy()

        # -----------------------------------------------------------------
        # Part 5 - Quantum Annealing

        # Initial guess
        s0 = np.ones(s1.shape) * 1 / 3500
        A = construct_Ad(M, R, 1)
        # L = max(abs(s1 - s0)) + 0.05 * max(abs(s1 - s0))
        L = 3e-5
        directory = f'{results_dir}/{layer}'
        os.makedirs(directory, exist_ok=True)

        for i in range(0, 10):
            b = (t + L * M @ I - M @ s0) / L

            Q = binary_least_squares_qubo(A, b)

            # Solve the QUBO using D-Wave's system
            sampler = EmbeddingComposite(DWaveSampler())
            sampleset = sampler.sample_qubo(Q, num_reads=100)

            # Get the best sample
            q = dict_to_vector_auto(sampleset.first.sample)
            x = binary2real(q, R, 2)

            # Update s
            s = s0 + L * (x - I)

            # Save results
            s0 = s
            L = L / 2
            filename = f'{directory}/s_{i}.txt'
            np.savetxt(filename, s)

        print(f'Layer {layer}: Max error =', max(abs(1 / s1 - 1 / s)))



In [6]:
quantum_annealing_inversion(noise_level=1e-2, results_dir='results-noise-1-constantL')

Process:   0%|          | 0/15 [00:00<?, ?it/s]

Process:   7%|▋         | 1/15 [01:07<15:39, 67.10s/it]

Layer 15: Max error = 6.436859988476044


Process:  13%|█▎        | 2/15 [02:14<14:37, 67.52s/it]

Layer 16: Max error = 6.674437022757957


...

Layer 27: Max error = 22.47869181453143


Process:  93%|█████████▎| 14/15 [16:10<01:05, 65.21s/it]

Layer 28: Max error = 42.06100758996172


Process: 100%|██████████| 15/15 [17:19<00:00, 69.29s/it]

Layer 29: Max error = 797.3134495051177


In [10]:
quantum_annealing_inversion(noise_level=2e-2, results_dir='results-noise-2-constantL')

Process:   7%|▋         | 1/15 [01:14<17:27, 74.79s/it]

Layer 0: Max error = 87.93432751564296


Process:  13%|█▎        | 2/15 [03:21<22:49, 105.35s/it]

Layer 1: Max error = 146.5693930023681


Process:  20%|██        | 3/15 [04:41<18:45, 93.82s/it] 

...

Layer 12: Max error = 15.146375858201736


Process:  93%|█████████▎| 14/15 [20:40<01:33, 93.82s/it]

Layer 13: Max error = 10.77533557991137


Process: 100%|██████████| 15/15 [22:07<00:00, 88.53s/it]

Layer 14: Max error = 16.789205886011132


In [11]:
quantum_annealing_inversion(noise_level=5e-2, results_dir='results-noise-5-constantL')

Process:   7%|▋         | 1/15 [01:24<19:40, 84.35s/it]

Layer 0: Max error = 239.5365256975333


Process:  13%|█▎        | 2/15 [02:45<17:48, 82.23s/it]

Layer 1: Max error = 297.2272367067071


Process:  20%|██        | 3/15 [04:14<17:04, 85.38s/it]

...

Layer 12: Max error = 34.056511365309234


Process:  93%|█████████▎| 14/15 [22:05<01:42, 102.68s/it]

Layer 13: Max error = 34.41339485470917


Process: 100%|██████████| 15/15 [23:32<00:00, 94.17s/it] 

Layer 14: Max error = 47.603964476554665


In [20]:
quantum_annealing_inversion(noise_level=5e-2, results_dir='results-noise-5')

Process:   3%|▎         | 1/30 [01:10<34:10, 70.70s/it]

Layer 0: Max error = 22.528672991102667


Process:   7%|▋         | 2/30 [02:19<32:22, 69.37s/it]

Layer 1: Max error = 47.5098298760372


Process:  10%|█         | 3/30 [03:24<30:24, 67.56s/it]

...

Layer 27: Max error = 64.06048282074107


Process:  97%|█████████▋| 29/30 [42:19<01:48, 108.80s/it]

Layer 28: Max error = 63.10374015818479


Process: 100%|██████████| 30/30 [43:59<00:00, 87.98s/it] 

Layer 29: Max error = 33.53641736081545
